In [1]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive. For more details see https://docs.quantum-computing.ibm.com/run/primitives
# result = Sampler().run(circuits).result()

In [13]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit.algorithms import VQE
from qiskit.algorithms.optimizers import SPSA, COBYLA
from qiskit.primitives import Estimator
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

### STEP-1: Setting up the Molecular System
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM)

problem = driver.run()

### STEP-2: Hamiltonian Setup (Second Quantization)
hamiltonian= problem.hamiltonian.second_q_op()

### STEP-3: Qubit Mapping
qubit_mapper= JordanWignerMapper()
qubit_hamiltonian= qubit_mapper.map(hamiltonian)

### STEP-4: Initial State Preparation
initial_state= HartreeFock(
    num_spatial_orbitals= problem.num_spatial_orbitals,
    num_particles= problem.num_particles,
    qubit_mapper= qubit_mapper)

### STEP-5: Ansatz Definition
ansatz = UCCSD(
    num_spatial_orbitals= problem.num_spatial_orbitals,
    num_particles= problem.num_particles,
    qubit_mapper= qubit_mapper,
    initial_state= initial_state)

### STEP-6: VQE Setup
optimizer= COBYLA (maxiter=1000)
backend= Aer.get_backend("statevector_simulator")
vqe= VQE(ansatz, optimizer, quantum_instance=backend)
result=vqe.compute_minimum_eigenvalue(qubit_hamiltonian)
print(result)

/tmp/ipykernel_777/2312310269.py:43: DeprecationWarning: The class ``qiskit.algorithms.minimum_eigen_solvers.vqe.VQE`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. Instead, use the class ``qiskit.algorithms.minimum_eigensolvers.VQE``. See https://qisk.it/algo_migration for a migration guide.
  vqe= VQE(ansatz, optimizer, quantum_instance=backend)


{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 49,
    'eigenstate': array([ 4.90653893e-17-3.46944695e-17j,  2.74763936e-16-3.02236498e-16j,
        1.24900090e-16-1.34398247e-16j,  2.45326947e-17-1.38777878e-17j,
        8.83178169e-17-1.70002901e-16j,  9.93763127e-01-3.79775472e-16j,
       -1.89665611e-06-9.81284559e-18j, -5.69064928e-17-3.19189120e-16j,
        1.14771975e-16+7.28583860e-17j, -4.99429076e-05+3.92553696e-17j,
       -1.11511639e-01-1.57829965e-17j,  3.43427144e-17+6.93889390e-17j,
        6.86915451e-17+6.93889390e-17j,  0.00000000e+00-7.84416158e-18j,
       -9.81305669e-17+2.98484317e-18j,  4.90653893e-18-1.38777878e-17j]),
    'eigenvalue': (-1.857275027209684+0j),
    'optimal_circuit': None,
    'optimal_parameters': {   ParameterVectorElement(t[0]): 1.8966561089365834e-06,
                              ParameterVectorElement(t[1]): 4.994290761633485e-05,
                              ParameterVectorElement(t[2]): -0.11174404667321236},
    '

In [3]:
optimal_params = result.optimal_point
print("Optimal Parameters:", optimal_params)

Optimal Parameters: [ 1.89665611e-06  4.99429076e-05 -1.11744047e-01]


Simulation Of Real Hardware

In [ ]:
from qiskit_ibm_runtime import Options, Estimator, Session
from qiskit_ibm_runtime import QiskitRuntimeService
import numpy as np

options=Options()
options.optimization_level=0
options.resilience_level=0
service= QiskitRuntimeService()
backend= service.get_backend('ibm_osaka')

with Session(service=service, backend= backend) as session:
    estimator= Estimator(options=options)
    cost_func= lambda params:estimator.run(ansatz, qubit_hamiltonian, parameter_values=params).result().values[0]
    result= optimizer.minimize(cost_func, x0= np.zeros(ansatz.num_parameters))
    print(result)

In [7]:
job = service.job("cp43bnp2zy6g0084j9cg")
job.status()

<JobStatus.DONE: 'job has successfully run'>

In [8]:
job.result()

EstimatorResult(values=array([-1.49445814]), metadata=[{'variance': 0.19888308261667137, 'shots': 4000}])

In [12]:
options_pec =Options()
options_pec.optimization_level=1
options_pec.resilience_level=3

service= QiskitRuntimeService()
backend= service.get_backend('ibm_brisbane')

with Session(service=service, backend= backend) as session:
    estimator= Estimator(options=options_pec)
    cost_func= lambda params:estimator.run(ansatz, qubit_hamiltonian, parameter_values=params).result().values[0]
    result= optimizer.minimize(cost_func, x0= np.zeros(ansatz.num_parameters))
    print(result)

base_primitive._run_primitive:INFO:2023-12-24 16:45:16,876: Submitting job using options {'optimization_level': 1, 'resilience_level': 3, 'max_execution_time': None, 'transpilation': {'skip_transpilation': False, 'initial_layout': None, 'layout_method': None, 'routing_method': None, 'approximation_degree': None, 'optimization_settings': {'level': 1}, 'coupling_map': None, 'basis_gates': None}, 'resilience': {'noise_amplifier': 'TwoQubitAmplifier', 'noise_factors': (1, 3, 5), 'extrapolator': 'LinearExtrapolator', 'level': 3}, 'execution': {'shots': 4000, 'init_qubits': True, 'noise_model': None, 'seed_simulator': None}, 'environment': {'log_level': 'WARNING', 'callback': None, 'job_tags': []}, 'simulator': {'noise_model': None, 'seed_simulator': None, 'coupling_map': None, 'basis_gates': None}}


Traceback (most recent call last):
  Cell In[12], line 11
    result= optimizer.minimize(cost_func, x0= np.zeros(ansatz.num_parameters))
  File /opt/conda/lib/python3.10/site-packages/qiskit/algorithms/optimizers/scipy_optimizer.py:149 in minimize
    raw_result = minimize(
  File /opt/conda/lib/python3.10/site-packages/scipy/optimize/_minimize.py:705 in minimize
    res = _minimize_cobyla(fun, x0, args, constraints, callback=callback,
  File /opt/conda/lib/python3.10/site-packages/scipy/optimize/_cobyla_py.py:34 in wrapper
    return func(*args, **kwargs)
  File /opt/conda/lib/python3.10/site-packages/scipy/optimize/_cobyla_py.py:273 in _minimize_cobyla
    xopt, info = cobyla.minimize(calcfc, m=m, x=np.copy(x0), rhobeg=rhobeg,
  File /opt/conda/lib/python3.10/site-packages/scipy/optimize/_cobyla_py.py:261 in calcfc
    f = fun(np.copy(x), *args)
  Cell In[12], line 10 in <lambda>
    cost_func= lambda params:estimator.run(ansatz, qubit_hamiltonian, parameter_values=params).result().v

capi_return is NULL
Call-back cb_calcfc_in__cobyla__user__routines failed.


In [5]:
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService()
 
service.backends()

[<IBMBackend('ibm_kyoto')>,
 <IBMBackend('ibm_osaka')>,
 <IBMBackend('ibmq_qasm_simulator')>,
 <IBMBackend('simulator_extended_stabilizer')>,
 <IBMBackend('simulator_mps')>,
 <IBMBackend('simulator_statevector')>,
 <IBMBackend('simulator_stabilizer')>,
 <IBMBackend('ibm_brisbane')>]

In [7]:
service.backends(simulator=False, operational=True, min_num_qubits=5)

[<IBMBackend('ibm_kyoto')>,
 <IBMBackend('ibm_osaka')>,
 <IBMBackend('ibm_brisbane')>]

In [8]:
service.least_busy(operational=True, min_num_qubits=5)

<IBMBackend('ibmq_qasm_simulator')>